# Purpose:
- For **Fire Perimeter**:
    - Generate a circular polygon and crop the flight image based on that polygon
- For **Burn Severity**:
    - Crop the raster image based on input fire perimeter in GeoJSON (*product generated* or [CALFIRE](https://gis.data.ca.gov/datasets/CALFIRE-Forestry::california-fire-perimeters-all-1/explore)) or shapefile ([MTBS](https://www.mtbs.gov/viewer/index.html))
- For **either**:
    - Reproject a raster image to another coordinate reference systems (Check for burn severity raster images [MTBS](https://www.mtbs.gov/viewer/index.html))
    
***Due to GitHub file size limit, the post-RTC data is not available on this repository. Please following the steps in the 'radiometric_terrain_correction' repository to obtain the RTC data***

Cropped RTC image is available for the application of the other 3 notebooks in this repo.


In [40]:
import sys
from pathlib import Path

# Add the path to the utils folder to sys.path
utils_path = Path('../python').resolve()
print(utils_path)
sys.path.append(str(utils_path))

from pathlib import Path
from rasterio.crs import CRS
from crop_utils import (crop_image_by_coordinates, 
                        crop_image_by_geojson_shp, 
                        reproject_geotiff)
from edit_path_utils import (edit_paths)

/Users/krismannino/Code/CADS/IMPACT/uavsar-wildfire/python


---
## Crop image by radius from center
Returns a cropped version of the input image based intersection of the specificed circle

**Parameters**:
- `path_to_images` (list): a list containing the paths to the images cropping from 
- `output_names` (list): a list containing the output names for the cropped images [**.tif** files]
- `center_longitude` (float): center longitude of the fire
- `center_latitude` (float): center latitude of the fire
- `radius_in_km` (float): radius from the coordinate in km to crop

In [41]:
data_dir = Path('/Volumes/BlackT7/bobcat_flight_paths')
tifs = sorted(list(data_dir.rglob('./*_rtc.grd')))
# tifs = sorted(list(data_dir.rglob('./*_test.tif')))

# adjust for wanted and unwanted tifs
include = [
            "181011",
            "210526"
          ]
exclude = [
            "26526"
          ]

tifs = edit_paths(include,exclude,tifs)
print(*tifs, sep="\n")

/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_18076_003_181011_L090_CX_01/SanAnd_08525_18076_003_181011_HVHV_rtc.grd
/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_21041_007_210526_L090_CX_01/SanAnd_08525_21041_007_210526_HVHV_rtc.grd


In [42]:
path_to_images = tifs
center_longitude = -117.92976
center_latitude = 34.33253
# radius_in_km = 25
width_km = 45
height_km =40
output_names = [str(data_dir) + '/' + file.stem + '_rectangle_cropped_' + str(width_km) +"km_"+ str(height_km) + 'km.tif' for file in path_to_images]
print(*output_names, sep="\n")##

/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_18076_003_181011_HVHV_rtc_rectangle_cropped_45km_40km.tif
/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_21041_007_210526_HVHV_rtc_rectangle_cropped_45km_40km.tif


---
Now perform the cropping and the output will be saved at the working directory. Or you can edit `output_names` variable to set a location

In [43]:
for i in range(len(path_to_images)):
    crop_image_by_coordinates(path_to_images[i],
                              output_names[i], 
                              center_longitude, 
                              center_latitude, 
                              width_km,
                              height_km)

/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_18076_003_181011_HVHV_rtc_rectangle_cropped_45km_40km.tif is outputted!
/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_21041_007_210526_HVHV_rtc_rectangle_cropped_45km_40km.tif is outputted!


In [44]:
##data_dir = Path('../data/latuna')
incs = sorted(list(data_dir.rglob('./*.inc')))

# adjust for wanted and unwanted inc flight paths
# include = [
#             "181011"
#           ]
# exclude = [
#             "hello"
#           ]

incs = edit_paths(include,exclude,incs)
print(*incs, sep="\n")

/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_18076_003_181011_L090_CX_01/SanAnd_08525_18076_003_181011_L090_CX_01.inc
/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_21041_007_210526_L090_CX_01/SanAnd_08525_21041_007_210526_L090_CX_01.inc


In [45]:
path_to_images = incs
##center_longitude = -118.2674
##center_latitude = 34.22957
##radius_in_km = 10
output_names = [str(data_dir) + '/' + file.stem  + '_inc_cropped_' + str(width_km) +"km_"+ str(height_km) + 'km.tif' for file in path_to_images]
print(*output_names, sep="\n")##

/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_18076_003_181011_L090_CX_01_inc_cropped_45km_40km.tif
/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_21041_007_210526_L090_CX_01_inc_cropped_45km_40km.tif


---
Now perform the cropping and the output will be saved at the working directory. Or you can edit `output_names` variable to set a location

In [46]:
for i in range(len(path_to_images)):
    crop_image_by_coordinates(path_to_images[i],
                              output_names[i], 
                              center_longitude, 
                              center_latitude, 
                              width_km,
                              height_km)

/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_18076_003_181011_L090_CX_01_inc_cropped_45km_40km.tif is outputted!
/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_21041_007_210526_L090_CX_01_inc_cropped_45km_40km.tif is outputted!


---
## Crop image by a GeoJSON or Shapefile
Returns a cropped version of the input image corresponding to the polygon(s) in the input GeoJSON or Shapefile.

**Parameters**:
- `path_to_polygon_file` (str): path to the geojson or shapefile
- `path_to_images` (list): a list containing the paths to the images cropping from
- `output_names` (list): a list containing the output names for the cropped images. [**.tif** files]

The sample below will be cropping with a shapefile

In [30]:
shp_dir = Path('/Volumes/BlackT7/bobcat_perimeter_wen')
# shps = sorted(list(shp_dir.glob('./*.shp')))
shps = sorted([shp for shp in shp_dir.glob('*.shp') if not shp.name.startswith('._')])
shps

[PosixPath('/Volumes/BlackT7/bobcat_perimeter_wen/bobcat_perimeter_wen.shp')]

In [31]:
##shps = '../data/bobcat/uavsar_perimeter/bobcat_perimeter_bilinear_inc_south.geojson'
shps = '/Volumes/BlackT7/bobcat_perimeter_wen/bobcat_perimeter_wen.shp'
shps

'/Volumes/BlackT7/bobcat_perimeter_wen/bobcat_perimeter_wen.shp'

In [43]:
data_dir = Path('/Volumes/BlackT7/bobcat_flight_paths')
# tifs = sorted(list(data_dir.glob('./*weighted*.tif')))

tifs = sorted(list(data_dir.glob('*.tif')))

# adjust for wanted and unwanted tifs
include = [
            "181011"
          ]
exclude = [
            # "35km"
          ]

tifs = edit_paths(include,exclude,tifs)
print(*tifs, sep="\n")

/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_18076_003_181011_HVHV_rtc_cropped_25km.tif
/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_18076_003_181011_HVHV_rtc_cropped_35km.tif
/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_18076_003_181011_L090_CX_01_inc_25km.tif
/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_18076_003_181011_L090_CX_01_inc_35km.tif
/Volumes/BlackT7/bobcat_flight_paths/SanAnd_26526_18076_002_181011_HVHV_rtc_cropped_25km.tif
/Volumes/BlackT7/bobcat_flight_paths/SanAnd_26526_18076_002_181011_HVHV_rtc_cropped_25km_shp_crop.tif
/Volumes/BlackT7/bobcat_flight_paths/SanAnd_26526_18076_002_181011_L090_CX_01_inc_25km.tif
/Volumes/BlackT7/bobcat_flight_paths/SanAnd_26526_18076_002_181011_L090_CX_01_inc_25km_shp_crop.tif


In [35]:
path_to_polygon_file = shps
path_to_images = tifs
output_names = [str(data_dir) + '/' + file.stem + '_shp_crop.tif' for file in tifs]
output_names

['/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_21041_007_210526_HVHV_rtc_cropped_25km_shp_crop.tif',
 '/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_21041_007_210526_L090_CX_01_inc_25km_shp_crop.tif']

---
Now perform the cropping and the output will be saved at the working directory. Or you can edit `output_names` variable to set a location

replace `path_to_polygon_file` with the geojson path if cropping by geojson

In [36]:
for i in range(len(path_to_images)):
    crop_image_by_geojson_shp(path_to_polygon_file,
                              path_to_images[i], 
                              output_names[i])

/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_21041_007_210526_HVHV_rtc_cropped_25km_shp_crop.tif is outputted.
/Volumes/BlackT7/bobcat_flight_paths/SanAnd_08525_21041_007_210526_L090_CX_01_inc_25km_shp_crop.tif is outputted.


---
## Reproject Raster image to another coordinate reference systems
Used to reproject the MTBS burn severity map from ESRI:102039 to EPSG:4326, crs of UAVSAR data

The output could be used to extract the burn severity map of a particular fire or area, using `crop_image_by_coordinates`, `crop_image_by_geojson`, or `crop_image_by_shp` from above

**Parameters**
- `target_crs` (CRS): the crs to reproject the image to
- `input_raster_path` (str): path to the original raster image
- `output_raster_path` (str): path for the reprojected raster image [**.tif** files]

In [ ]:
data_dir = Path('/Volumes/BlueT7/test_bob/')
tifs = sorted(list(data_dir.glob('./*.tif')))
tifs

In [ ]:
target_crs = CRS.from_epsg(4326)   
input_raster_path = tifs[0]
output_raster_path = 'mtbs_CA_2009_crs4326.tif'

---
Now perform the cropping and the output will be saved at the working directory. Or you can edit `output_names` variable to set a location

In [ ]:
reproject_geotiff(target_crs, input_raster_path, output_raster_path)